# Créer son propre modèle ChatGPT

Ce code est disponible dans le repo du livre Generative Deep Learning de David Foster disponible [ici](https://github.com/davidADSP/Generative_Deep_Learning_2nd_Edition/blob/main/notebooks/09_transformer/gpt/gpt.ipynb)


Vous pouvez également avoir un code disponible [ici](https://keras.io/examples/generative/text_generation_with_miniature_gpt/)

# Générateur de Descriptions de Vin 🍷



## Objectif 🎯

- Concevoir et entraîner un modèle d'intelligence artificielle pour générer des descriptions de vin qui capturent l'essence, les arômes, les saveurs et le caractère unique de chaque bouteille.

## Données 📊

- Source : - Source : [winemag-data-130k-v2.json sur Kaggle](https://www.kaggle.com/zynicide/wine-reviews)

- Plus de 130 000 avis sur les vins collectés par *Wine Magazine*.
- Chaque entrée contient des informations telles que le pays, la désignation, le prix, la variété et, le plus important pour notre projet, une description détaillée du vin.

## Méthodologie ⚙️

1. **Nettoyage des données** : Élimination des doublons, traitement des valeurs manquantes, et préparation des descriptions pour l'entraînement.
2. **Entraînement du modèle Transformer** : Utilisation d'une architecture Transformer pour apprendre les nuances des descriptions de vin et générer de nouvelles descriptions.



In [41]:
%load_ext autoreload
%autoreload 2
import numpy as np
import json
import re
import string
from IPython.display import display, HTML

import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0. Paramètres du projet <a name="parameters"></a>

In [2]:
# Taille du vocabulaire utilisé pour le modèle (nombre de mots uniques pris en compte)
VOCAB_SIZE = 10000

# Longueur maximale des séquences traitées par le modèle
MAX_LEN = 80

# Dimension de l'embedding des mots (représentation vectorielle des mots)
EMBEDDING_DIM = 256

# Dimension des clés pour l'attention multi-têtes
KEY_DIM = 256

# Nombre de "têtes" dans l'attention multi-têtes
N_HEADS = 2

# Dimension du réseau feed-forward dans le Transformer
FEED_FORWARD_DIM = 256

# Fraction des données utilisée pour la validation pendant l'entraînement
VALIDATION_SPLIT = 0.2

# Graine aléatoire pour la reproductibilité
SEED = 42

# Si True, charge un modèle pré-entraîné au lieu de former un nouveau modèle
LOAD_MODEL = True

# Taille des lots utilisés pendant l'entraînement
BATCH_SIZE = 32

# Nombre de cycles complets de passage sur l'ensemble de données pendant l'entraînement
EPOCHS = 5


## 1. Load the data <a name="load"></a>

In [42]:
# Charger l'ensemble de données complet
with open("winemag-data-130k-v2.json") as json_data:
    wine_data = json.load(json_data)


In [44]:
wine_data[10]

{'points': '87',
 'title': 'Kirkland Signature 2011 Mountain Cuvée Cabernet Sauvignon (Napa Valley)',
 'description': 'Soft, supple plum envelopes an oaky structure in this Cabernet, supported by 15% Merlot. Coffee and chocolate complete the picture, finishing strong at the end, resulting in a value-priced wine of attractive flavor and immediate accessibility.',
 'taster_name': 'Virginie Boone',
 'taster_twitter_handle': '@vboone',
 'price': 19,
 'designation': 'Mountain Cuvée',
 'variety': 'Cabernet Sauvignon',
 'region_1': 'Napa Valley',
 'region_2': 'Napa',
 'province': 'California',
 'country': 'US',
 'winery': 'Kirkland Signature'}

In [45]:
# Filtrer l'ensemble de données
filtered_data = [
    "wine review : "
    + x["country"]
    + " : "
    + x["province"]
    + " : "
    + x["variety"]
    + " : "
    + x["description"]
    for x in wine_data
    if x["country"] is not None
    and x["province"] is not None
    and x["variety"] is not None
    and x["description"] is not None
]

In [6]:
filtered_data

["wine review : Italy : Sicily & Sardinia : White Blend : Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",
 "wine review : Portugal : Douro : Portuguese Red : This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.",
 'wine review : US : Oregon : Pinot Gris : Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.',
 'wine review : US : Michigan : Riesling : Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.',
 "wine r

In [46]:
# Compter le nombre de description de vin
n_wines = len(filtered_data)
print(f"{n_wines} description chargées")


129907 description chargées


In [47]:
example = filtered_data[25]
print(example)

wine review : US : California : Pinot Noir : Oak and earth intermingle around robust aromas of wet forest floor in this vineyard-designated Pinot that hails from a high-elevation site. Small in production, it offers intense, full-bodied raspberry and blackberry steeped in smoky spice and smooth texture.


##  Tokeniser les données <a name="tokenize"></a>


La **tokenisation** est une étape fondamentale du traitement du langage naturel (NLP). Elle consiste à diviser une chaîne de texte en unités plus petites, appelées "tokens". Ces tokens peuvent être aussi simples que des mots ou aussi complexes que des phrases entières.






In [48]:
def pad_punctuation(s):
    """
    Ajoute des espaces autour de la ponctuation pour les traiter comme des 'mots' séparés.
    
    Args:
        s (str): Chaîne de caractères à traiter.
        
    Returns:
        str: Chaîne de caractères avec la ponctuation séparée par des espaces.
    """
    
    # Remplacer chaque caractère de ponctuation par lui-même entouré d'espaces
    s = re.sub(f"([{string.punctuation}, '\n'])", r" \1 ", s)
    
    # Remplacer les espaces multiples par un seul espace
    s = re.sub(" +", " ", s)
    
    return s

# Appliquer la fonction `pad_punctuation` à chaque élément de `filtered_data`
text_data = [pad_punctuation(x) for x in filtered_data]


In [49]:
# Afficher une description
example_data = text_data[30]
example_data

'wine review : France : Beaujolais : Gamay : Red cherry fruit comes laced with light tannins , giving this bright wine an open , juicy character . '

In [50]:
# Convertir en un Dataset TensorFlow
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)  # Créer un dataset TensorFlow à partir de la liste `text_data`
    .batch(BATCH_SIZE)  # Grouper les données en lots de taille `BATCH_SIZE`
    .shuffle(1000)  # Mélanger les données avec un buffer de taille 1000
)


In [51]:
# Créer une couche de vectorisation
vectorize_layer = layers.TextVectorization(
    standardize="lower",                   # Standardiser le texte en le mettant en minuscules
    max_tokens=VOCAB_SIZE,                 # Nombre maximal de tokens uniques dans le vocabulaire
    output_mode="int",                     # Mode de sortie où chaque token est représenté par un entier unique
    output_sequence_length=MAX_LEN + 1,    # Longueur de sortie pour chaque séquence vectorisée
)


In [52]:
# Adapter la couche aux données d'entraînement
vectorize_layer.adapt(text_ds)

# Récupérer le vocabulaire généré par la couche de vectorisation
vocab = vectorize_layer.get_vocabulary()
vocab

['',
 '[UNK]',
 ':',
 ',',
 '.',
 'and',
 'the',
 'wine',
 'a',
 'of',
 'review',
 'with',
 'this',
 'is',
 '-',
 'it',
 'flavors',
 'in',
 "'",
 'to',
 'us',
 's',
 'fruit',
 'on',
 'red',
 'that',
 'aromas',
 'blend',
 'palate',
 'california',
 'acidity',
 'finish',
 'from',
 'but',
 'tannins',
 'drink',
 'cherry',
 'black',
 'ripe',
 'are',
 'italy',
 'has',
 'france',
 'pinot',
 'sauvignon',
 'cabernet',
 'for',
 'by',
 '%',
 'spice',
 'notes',
 'as',
 'an',
 'white',
 'its',
 'oak',
 'fresh',
 'rich',
 'dry',
 'berry',
 'nose',
 'chardonnay',
 'noir',
 'style',
 'full',
 'bordeaux',
 'now',
 'plum',
 'soft',
 'apple',
 'fruits',
 'well',
 'sweet',
 'crisp',
 'blackberry',
 'offers',
 'light',
 'texture',
 'dark',
 'there',
 'while',
 'citrus',
 'bodied',
 'shows',
 'through',
 'spain',
 'vanilla',
 'bright',
 'at',
 'very',
 'pepper',
 'more',
 'syrah',
 'juicy',
 'green',
 'lemon',
 'fruity',
 'raspberry',
 'valley',
 'good',
 'merlot',
 'blanc',
 'firm',
 'washington',
 'some',


In [53]:
# Afficher quelques correspondances entre tokens et mots
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")


0: 
1: [UNK]
2: :
3: ,
4: .
5: and
6: the
7: wine
8: a
9: of


In [54]:
# Afficher le même exemple converti en entiers
example_tokenised = vectorize_layer(example_data)

# Afficher la séquence d'entiers résultante
print(example_tokenised.numpy())


[  7  10   2  42   2 580   2 620   2  24  36  22 287 742  11  76  34   3
 429  12  87   7  52 456   3  93 110   4   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0]


## Créer l'ensemble d'entraînement <a name="create"></a>

In [55]:
def prepare_inputs(text):
    """
    Prépare les entrées pour l'entraînement en créant des paires de phrases : 
    l'une avec le texte original et l'autre avec le texte décalé d'un mot.
    
    Args:
        text (tf.Tensor): Tensor contenant le texte brut.
        
    Returns:
        tuple: Deux tensors, l'un avec le texte original (x) et l'autre avec le texte décalé d'un mot (y).
    """
    
    # Étendre les dimensions du texte pour le traitement
    text = tf.expand_dims(text, -1)
    
    # Convertir le texte en séquences d'entiers
    tokenized_sentences = vectorize_layer(text)
    
    # x contient tous les mots sauf le dernier de chaque phrase
    x = tokenized_sentences[:, :-1]
    
    # y contient tous les mots sauf le premier de chaque phrase, décalant ainsi les séquences d'un mot
    y = tokenized_sentences[:, 1:]
    
    return x, y

# Appliquer la fonction `prepare_inputs` à l'ensemble de données
train_ds = text_ds.map(prepare_inputs)


In [56]:
example_input_output = train_ds.take(1).get_single_element()

In [57]:
# exemple
example_input_output[0][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([   7,   10,    2,   20,    2,   29,    2,  557,    2, 6399,  307,
          3,  127,   95,    3, 1776,  151,    5,   49,    3,   12,  557,
         13, 1028,  299,    5, 1344,  282,    3,   52,  575, 1969,   19,
        123,  467,   33,  139,    6, 4855,    9,   16, 1461,  453,    8,
       7678, 2639,    4,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

In [19]:
# Exemple de sortie (décalé d'un token)
example_input_output[1][0]


<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  10,    2,   20,    2,   29,    2,  286,    2,    8,  335, 2137,
          9, 6151,  470,    3,   12,   57,   63,    9,  164,   83,   24,
         36,    3,  213,    3,  276,  846,    5,  685,    3,  151,  859,
         14,  111,   26,   23,    6,   60,    4,   38,   67,    5,  204,
         16,  793,    6,   28,    3, 2171,   47,  719,  333,    5,  266,
          9,  522,  841,    4,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

##  Créer la fonction de masquage pour l'attention causale <a name="causal"></a>

In [20]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    Crée un masque pour l'attention causale.
    
    Ce masque permet de s'assurer que lors de la prédiction d'un token, 
    seul le passé (les tokens précédents) est pris en compte.
    
    Args:
        batch_size (int): Taille du lot.
        n_dest (int): Nombre de tokens de destination.
        n_src (int): Nombre de tokens source.
        dtype (tf.DType): Type des éléments du masque.
        
    Returns:
        tf.Tensor: Masque d'attention causale de forme [batch_size, n_dest, n_src].
    """
    
    # Crée des indices pour les tokens de destination et source
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    
    # Détermine la relation entre les indices de destination et de source
    m = i >= j - n_src + n_dest
    
    # Convertit le masque booléen en type spécifié
    mask = tf.cast(m, dtype)
    
    # Redimensionne le masque pour le rendre compatible avec les dimensions attendues
    mask = tf.reshape(mask, [1, n_dest, n_src])
    
    # Duplique le masque pour tout le lot
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)

# Affiche le masque d'attention causale transposé pour une meilleure visualisation
np.transpose(causal_attention_mask(1, 10, 10, dtype=tf.int32)[0])


array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

## Créer une couche de bloc Transformer<a name="transformer"></a>

In [21]:
class TransformerBlock(layers.Layer):
    """
    Couche de bloc Transformer.
    
    Ce bloc est composé d'une attention multi-têtes suivie d'une 
    normalisation de couche et d'un réseau feed-forward.
    
    Attributes:
        num_heads (int): Nombre de têtes pour l'attention multi-têtes.
        key_dim (int): Dimension de la clé pour l'attention.
        embed_dim (int): Dimension de l'embedding.
        ff_dim (int): Dimension du réseau feed-forward interne.
        dropout_rate (float, optional): Taux de dropout. Par défaut à 0.1.
    """

    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate

        # Initialisation des couches
        self.attn = layers.MultiHeadAttention(
            num_heads, key_dim, output_shape=embed_dim
        )
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        """Passage en avant du bloc Transformer."""
        
        # Calcul du masque d'attention causale
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(
            batch_size, seq_len, seq_len, tf.bool
        )

        # Application de l'attention multi-têtes
        attention_output, attention_scores = self.attn(
            inputs,
            inputs,
            attention_mask=causal_mask,
            return_attention_scores=True,
        )
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)

        # Application du réseau feed-forward
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        return (self.ln_2(out1 + ffn_output), attention_scores)

    def get_config(self):
        """Retourne la configuration du bloc Transformer."""
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "dropout_rate": self.dropout_rate,
            }
        )
        return config


## Créer l'Embedding de Tokens et de Position <a name="embedder"></a>

In [22]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_len": self.max_len,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

## Construire le modèle Transformer <a name="transformer_decoder"></a>

In [23]:
inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(
    N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM
)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
gpt = models.Model(inputs=inputs, outputs=[outputs, attention_scores])
gpt.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), None])

In [24]:
gpt.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddi  (None, None, 256)         2580480   
 ng (TokenAndPositionEmbedd                                      
 ing)                                                            
                                                                 
 transformer_block (Transfo  ((None, None, 256),       658688    
 rmerBlock)                   (None, 2, None, None))             
                                                                 
 dense_2 (Dense)             (None, None, 10000)       2570000   
                                                                 
Total params: 5809168 (22.16 MB)
Trainable params: 5809168 (22.16 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [25]:
if LOAD_MODEL:
    # model.load_weights('./models/model')
    gpt = models.load_model("./models/gpt", compile=True)

## Entraîner le Transformer <a name="train"></a>

In [26]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature):#temperature: parametre pour controler l'alea
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y, att = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append(
                {
                    "prompt": start_prompt,
                    "word_probs": probs,
                    "atts": att[0, :, -1, :],
                }
            )
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("wine review", max_tokens=80, temperature=1.0)

In [27]:
# Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.ckpt",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [28]:
gpt.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/5
4060/4060 [==============================] - ETA: 0s - loss: 1.7996 - dense_2_loss: 1.7996
generated text:
wine review : chile : casablanca valley : syrah : excellent things off - value , with spice notes , and it feels fine , and the rest of the movement in leyda valley . this mixes briny , briny and briny flavors of citrus and berry . its finish is fresh but limited . 

4060/4060 [==============================] - 660s 162ms/step - loss: 1.7996 - dense_2_loss: 1.7996
Epoch 2/5
4060/4060 [==============================] - ETA: 0s - loss: 1.7775 - dense_2_loss: 1.7775
generated text:
wine review : spain : northern spain : verdejo : verdejo can [UNK] particular fruit like mas ' s sugar , enough on the nose , then limited in scope . the palate is soft , round and easy , while the finish is fresh and better than past vintages but underwhelming . 

4060/4060 [==============================] - 648s 160ms/step - loss: 1.7775 - dense_2_loss: 1.7775
Epoch 3/5
4060/4060 [=============

In [29]:
# Save the final model
gpt.save("./models/gpt")

INFO:tensorflow:Assets written to: ./models/gpt/assets


INFO:tensorflow:Assets written to: ./models/gpt/assets


# Générer du texte à l'aide du Transformer

In [30]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        highlighted_text = []
        for word, att_score in zip(
            i["prompt"].split(), np.mean(i["atts"], axis=0)
        ):
            highlighted_text.append(
                '<span style="background-color:rgba(135,206,250,'
                + str(att_score / max(np.mean(i["atts"], axis=0)))
                + ');">'
                + word
                + "</span>"
            )
        highlighted_text = " ".join(highlighted_text)
        display(HTML(highlighted_text))

        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [59]:
info = text_generator.generate(
    "wine review : fr", max_tokens=80, temperature=1.0
)


generated text:
wine review : fr : oregon : pinot noir : all [UNK] is billed as a to the wood lover ' s wine , without elegance . the brothers have been ponzi in their bottles of his winemaking , and the acidity called [UNK] , the wine sources that most of this dundee vineyard ' s overall reserve wine to add power . 



In [32]:
info = text_generator.generate(
    "wine review : italy", max_tokens=80, temperature=0.5
)


generated text:
wine review : italy : tuscany : sangiovese : this opens with a fragrance of wild berry , leather , menthol and a whiff of leather . the palate delivers black cherry , espresso and star anise alongside firm tannins and bright acidity . drink through 2019 . 



In [33]:
info = text_generator.generate(
    "wine review : germany", max_tokens=80, temperature=0.5
)
print_probs(info, vocab)


generated text:
wine review : germany : rheingau : riesling : a hint of mineral and crushed stone lend a savory tone to this dry , full - bodied wine . it ' s crisp and bright on the palate , with bright acidity and a light touch of white peach and nectarine flavors . drink now . 



::   	100.0%
-:   	0.0%
,:   	0.0%
grosso:   	0.0%
and:   	0.0%
--------



mosel:   	58.67%
rheingau:   	26.67%
rheinhessen:   	11.72%
pfalz:   	2.34%
nahe:   	0.28%
--------



::   	100.0%
-:   	0.0%
other:   	0.0%
grosso:   	0.0%
neagra:   	0.0%
--------



riesling:   	99.65%
pinot:   	0.23%
spätburgunder:   	0.11%
rosé:   	0.0%
white:   	0.0%
--------



...

.:   	97.38%
,:   	1.4%
that:   	1.21%
on:   	0.01%
finish:   	0.0%
--------



drink:   	43.42%
it:   	23.32%
:   	20.59%
finishes:   	9.64%
the:   	2.48%
--------



now:   	99.95%
it:   	0.03%
up:   	0.01%
now–2017:   	0.0%
soon:   	0.0%
--------



.:   	76.18%
through:   	23.44%
and:   	0.14%
for:   	0.09%
to:   	0.05%
--------



:   	100.0%
drink:   	0.0%
it:   	0.0%
the:   	0.0%
enjoy:   	0.0%
--------



In [34]:
info = text_generator.generate(
    "wine review : france", max_tokens=80, temperature=0.5
)
print_probs(info, vocab)


generated text:
wine review : france : bordeaux : bordeaux - style red blend : this is a ripe , fruity wine that has attractive black - currant flavors and a soft texture . the tannins are cushioned by the rich texture of the wine , giving a wine that is already delicious . drink from 2017 . 



::   	100.0%
grosso:   	0.0%
,:   	0.0%
other:   	0.0%
and:   	0.0%
--------



alsace:   	55.36%
bordeaux:   	24.58%
burgundy:   	9.27%
loire:   	3.17%
southwest:   	2.77%
--------



::   	100.0%
-:   	0.0%
.:   	0.0%
,:   	0.0%
other:   	0.0%
--------



bordeaux:   	98.32%
rosé:   	1.57%
merlot:   	0.06%
sauvignon:   	0.03%
cabernet:   	0.01%
--------



...

drink:   	99.31%
:   	0.43%
it:   	0.16%
the:   	0.09%
a:   	0.0%
--------



from:   	97.99%
now:   	1.79%
starting:   	0.19%
this:   	0.03%
soon:   	0.0%
--------



2017:   	68.59%
2018:   	17.3%
2016:   	7.52%
2015:   	2.07%
2019:   	1.78%
--------



.:   	100.0%
and:   	0.0%
to:   	0.0%
,:   	0.0%
for:   	0.0%
--------



:   	100.0%
the:   	0.0%
it:   	0.0%
andré:   	0.0%
george:   	0.0%
--------



In [40]:
info = text_generator.generate(
    "wine review : england", max_tokens=90, temperature=0.5
)
print_probs(info, vocab)


generated text:
wine review : england : england : chardonnay : a touch of apple scent the nose . the palate is all [UNK] . the same , however , the result is a slender , taut and slender , dry and whistle - clean , refreshing and the palate is whistle - clean , pure lemon and lime zest . there are glimpses of white grapefruit and quince , while the finish is persistent . 



::   	100.0%
-:   	0.0%
and:   	0.0%
grosso:   	0.0%
,:   	0.0%
--------



england:   	100.0%
france:   	0.0%
us:   	0.0%
central:   	0.0%
alsace:   	0.0%
--------



::   	100.0%
and:   	0.0%
-:   	0.0%
,:   	0.0%
generosity:   	0.0%
--------



sparkling:   	53.02%
chardonnay:   	27.58%
pinot:   	17.35%
riesling:   	0.73%
sauvignon:   	0.43%
--------



...

finish:   	95.27%
midpalate:   	2.27%
body:   	0.72%
palate:   	0.23%
texture:   	0.22%
--------



is:   	99.69%
provides:   	0.08%
makes:   	0.04%
remains:   	0.03%
does:   	0.02%
--------



whistle:   	73.23%
long:   	5.84%
clean:   	5.72%
dry:   	5.2%
[UNK]:   	1.17%
--------



and:   	54.21%
.:   	18.12%
,:   	17.14%
but:   	10.36%
with:   	0.09%
--------



:   	88.31%
drink:   	11.25%
this:   	0.29%
it:   	0.05%
the:   	0.03%
--------



In [60]:
import tensorflow as tf
import math

# Initialisation
sentence = "Salut, comment vas-tu ?"
embedding_layer = tf.keras.layers.Embedding(5000, 256)  # Exemple de dimension
tokenized_sentence = [1, 34, 56, 78, 3]
embedded_sentence = embedding_layer(tf.convert_to_tensor([tokenized_sentence]))

# Encodeur: Calcul des états de l'encodeur
encoder_rnn = tf.keras.layers.SimpleRNN(256, return_sequences=True)
encoder_states = encoder_rnn(embedded_sentence)

# Query, Key, Value pour l'encodeur
Q = tf.keras.layers.Dense(256, name="query")(encoder_states)
K = tf.keras.layers.Dense(256, name="key")(encoder_states)
V = tf.keras.layers.Dense(256, name="value")(encoder_states)

# Attention dans l'encodeur
QK = tf.matmul(Q, K, transpose_b=True)
QK_normalized = QK / tf.math.sqrt(tf.cast(256, tf.float32))
softmax = tf.nn.softmax(QK_normalized)
attention_encoder = tf.matmul(softmax, V)

# Décodeur: Supposons qu'il y ait une réponse que le décodeur essaie de générer
decoder_sentence = "Je vais bien, merci."  # Exemple de réponse
tokenized_decoder_sentence = [2, 35, 57, 79, 4]
embedded_decoder_sentence = embedding_layer(tf.convert_to_tensor([tokenized_decoder_sentence]))

# Calcul des états du décodeur
decoder_rnn = tf.keras.layers.SimpleRNN(256, return_sequences=True)
decoder_states = decoder_rnn(embedded_decoder_sentence)

# Query pour le décodeur et Key, Value de l'encodeur pour l'attention
Q_decoder = tf.keras.layers.Dense(256, name="query_decoder")(decoder_states)

# Attention dans le décodeur
QK_decoder = tf.matmul(Q_decoder, K, transpose_b=True)
QK_normalized_decoder = QK_decoder / tf.math.sqrt(tf.cast(256, tf.float32))
softmax_decoder = tf.nn.softmax(QK_normalized_decoder)
attention_decoder = tf.matmul(softmax_decoder, V)

print(attention_decoder)


tf.Tensor(
[[[ 0.05007175 -0.00486111  0.02492413 ...  0.00042409 -0.0139338
   -0.00778311]
  [ 0.05011487 -0.00488082  0.02498374 ...  0.00042968 -0.0139846
   -0.0077955 ]
  [ 0.05002842 -0.00485839  0.02475856 ...  0.00043114 -0.01390447
   -0.00771669]
  [ 0.05012161 -0.00482507  0.02490799 ...  0.00038137 -0.01394735
   -0.00781874]
  [ 0.05009252 -0.00488605  0.02477339 ...  0.0004489  -0.01395305
   -0.00768763]]], shape=(1, 5, 256), dtype=float32)
